In [1]:
# 강의자료의 Greeter 컨트랙을 수정, 배포하고 API를 사용하는 프로그램을 작성하세요.

# - Ganache에서 실행한다.

# - Greeter 컨트랙은 counter를 가지도록 하여 (Counter를 참조하여 수정한다), greet() 할 경우 몇 회나 호출되었는지 추적한다.

# - greet()를 호출하여 Hello를 출력하면서, 현재의 호출회수도 병기되도록 한다.

# - 3회 greet()를 호출하여 counter가 증가하는지 확인하면서 출력한다.

In [2]:
%%writefile src/Greeter.sol
// SPDX-License-Identifier: GPL-3.0

pragma solidity ^0.8.0;

contract Greeter {
    string greeting = "Hello";
    uint256 counter = 0;

    function greet() public returns (string memory, uint256) {
        counter++;
        return (greeting, counter);
    }
}

Overwriting src/Greeter.sol


In [3]:
!solc-windows.exe --optimize --combined-json abi,bin src/Greeter.sol > src/Greeter.json

In [4]:
!type src\Greeter.json

{"contracts":{"src/Greeter.sol:Greeter":{"abi":[{"inputs":[],"name":"greet","outputs":[{"internalType":"string","name":"","type":"string"},{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"}],"bin":"60c0604052600560809081526448656c6c6f60d81b60a0525f9061002390826100d1565b505f600155348015610033575f80fd5b50610190565b634e487b7160e01b5f52604160045260245ffd5b600181811c9082168061006157607f821691505b60208210810361007f57634e487b7160e01b5f52602260045260245ffd5b50919050565b601f8211156100cc57805f5260205f20601f840160051c810160208510156100aa5750805b601f840160051c820191505b818110156100c9575f81556001016100b6565b50505b505050565b81516001600160401b038111156100ea576100ea610039565b6100fe816100f8845461004d565b84610085565b602080601f831160018114610131575f841561011a5750858301515b5f19600386901b1c1916600185901b178555610188565b5f85815260208120601f198616915b8281101561015f57888601518255948401946001909101908401610140565b508582101561017c57878501515f19600388901b60f8

In [5]:
%%writefile src/GreeterDeployFromFile.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8345');

_abiBinJson = require('./Greeter.json');
contractName = Object.keys(_abiBinJson.contracts);
_abi = _abiBinJson.contracts[contractName].abi;
_bin = _abiBinJson.contracts[contractName].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    var deployed = await new web3.eth.Contract(_abi)
        .deploy({data: '0x'+_bin})
        .send({from: accounts[0], gas:364124})
        .then(function(newContractInstance) {
            console.log('address: ', newContractInstance.options.address);
        });
}

deploy();

Overwriting src/GreeterDeployFromFile.js


In [6]:
!node src/GreeterDeployFromFile.js

address:  0xCf966Bc401F63C48BA01805B764b1fbb7D12b028


In [7]:
%%writefile src/GreeterUseFromFile.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8345');

_abiBinJson = require('./Greeter.json');
contractName = Object.keys(_abiBinJson.contracts);
abi = _abiBinJson.contracts[contractName].abi;
addr = '0xCf966Bc401F63C48BA01805B764b1fbb7D12b028';

var greeter = new web3.eth.Contract(abi, addr);

async function doIt() {
    const accounts = await web3.eth.getAccounts();

    //await greeter.methods.greet().send({from: accounts[0], gas:100000});

    await greeter.methods.greet().call().then(function(result) {console.log(result[0], result[1])});
    await greeter.methods.greet().call().then(function(result) {console.log(result[0], result[1])});
    await greeter.methods.greet().call().then(function(result) {console.log(result[0], result[1])});
}
doIt();

Overwriting src/GreeterUseFromFile.js


In [8]:
!node src/GreeterUseFromFile.js

Hello 1
Hello 1
Hello 1


In [9]:
# 위의 결과를 보면 호출해도 counter가 증가하지 않는 것을 볼 수 있다.
# 이유는 call()함수는 컨트랙의 상태를 변경하지 않고 호출하는 함수이므로 값을 증가시키지 않기 때문이다.
# 만약 값을 증가시키려면 send() 함수를 이용해야 한다.

In [10]:
%%writefile src/GreeterUseFromFile.js
var Web3 = require('web3');
var web3 = new Web3('http://localhost:8345');

_abiBinJson = require('./Greeter.json');
contractName = Object.keys(_abiBinJson.contracts);
abi = _abiBinJson.contracts[contractName].abi;
addr = '0xCf966Bc401F63C48BA01805B764b1fbb7D12b028';

var greeter = new web3.eth.Contract(abi, addr);

async function doIt() {
    const accounts = await web3.eth.getAccounts();

    await greeter.methods.greet().send({from: accounts[0], gas:100000});
    await greeter.methods.greet().send({from: accounts[0], gas:100000});
    await greeter.methods.greet().send({from: accounts[0], gas:100000});
    await greeter.methods.greet().call().then(function(result) {console.log(result[0], result[1])});
}
doIt();

Overwriting src/GreeterUseFromFile.js


In [11]:
!node src/GreeterUseFromFile.js

Hello 4


In [12]:
# send()함수가 3번 실행되고 call() 함수가 1번 실행되면서 4가 출력된 것을 볼 수 있다.